# [Advent of Code 2020: Day 7](https://adventofcode.com/2020/day/7)

## \-\-\- Day 7: Handy Haversacks \-\-\-

You land at the regional airport in time for your next flight. In fact, it looks like you'll even have time to grab some food: all flights are currently delayed due to *issues in luggage processing*.

Due to recent aviation regulations, many rules (your puzzle input) are being enforced about bags and their contents; bags must be color\-coded and must contain specific quantities of other color\-coded bags. Apparently, nobody responsible for these regulations considered how long they would take to enforce!

For example, consider the following rules:

```
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.

```

These rules specify the required contents for 9 bag types. In this example, every `faded blue` bag is empty, every `vibrant plum` bag contains 11 bags (5 `faded blue` and 6 `dotted black`), and so on.

You have a **`shiny gold`** bag. If you wanted to carry it in at least one other bag, how many different bag colors would be valid for the outermost bag? (In other words: how many colors can, eventually, contain at least one `shiny gold` bag?)

In the above rules, the following options would be available to you:

*   A `bright white` bag, which can hold your `shiny gold` bag directly.
*   A `muted yellow` bag, which can hold your `shiny gold` bag directly, plus some other bags.
*   A `dark orange` bag, which can hold `bright white` and `muted yellow` bags, either of which could then hold your `shiny gold` bag.
*   A `light red` bag, which can hold `bright white` and `muted yellow` bags, either of which could then hold your `shiny gold` bag.

So, in this example, the number of bag colors that can eventually contain at least one `shiny gold` bag is **`4`**.

**How many bag colors can eventually contain at least one `shiny gold` bag?** (The list of rules is quite long; make sure you get all of it.)

In [1]:
import unittest
import re
from IPython.display import Markdown, display

from aoc_puzzle import AocPuzzle

class LuggageProcessing(AocPuzzle):
    
    def parse_data(self, raw_data):
        rules_list = raw_data.split('\n')
        self.data = {}
        
        for rule in rules_list:
            
            m = re.match('(\w+\s\w+)\sbags\scontain\s(.*)', rule)
            bag_color = m.group(1)
            bag_capacity = m.group(2)
            
            if bag_color not in self.data:                
                self.data[bag_color] = {}
                
            bag_cap_list = bag_capacity.split(', ')
            
            for bag_cap in bag_cap_list:
                m = re.match('(\d+)\s(\w+\s\w+)', bag_cap)
                if m:
                    bag_c = m.group(2)
                    count = m.group(1)
                    self.data[bag_color][bag_c] = int(count)
                    
    def get_all_bag_types(self, bag_color):
        """A recursive function to get all bag types contained in a single bag"""
        bag_contents = self.data[bag_color]
        all_contents = bag_contents.copy()
        if bag_contents:
            for bag in bag_contents:
                all_contents.update(self.data[bag])
                all_contents.update(self.get_all_bag_types(bag))
        return all_contents

    def get_bag_host_count(self, target_bag, output=False):
        result = 0
        for bag_color in self.data:
            all_contents = self.get_all_bag_types(bag_color)
            if target_bag in all_contents:
                result += 1
            
        if output:
            display(Markdown(f'### A `{target_bag}` bag can be in `{result}` other bags'))            
        return result
        

class TestBasic(unittest.TestCase):
    
    input_data = 'light red bags contain 1 bright white bag, 2 muted yellow bags.\ndark orange bags contain 3 bright white bags, 4 muted yellow bags.\nbright white bags contain 1 shiny gold bag.\nmuted yellow bags contain 2 shiny gold bags, 9 faded blue bags.\nshiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.\ndark olive bags contain 3 faded blue bags, 4 dotted black bags.\nvibrant plum bags contain 5 faded blue bags, 6 dotted black bags.\nfaded blue bags contain no other bags.\ndotted black bags contain no other bags.'

    def test_parse_data(self):
        exp_out = {'bright white' : {'shiny gold': 1},
                   'dark olive'   : {'dotted black': 4, 'faded blue': 3},
                   'dark orange'  : {'bright white': 3, 'muted yellow': 4},
                   'dotted black' : {},
                   'faded blue'   : {},
                   'light red'    : {'bright white': 1, 'muted yellow': 2},
                   'muted yellow' : {'faded blue': 9, 'shiny gold': 2},
                   'shiny gold'   : {'dark olive': 1, 'vibrant plum': 2},
                   'vibrant plum' : {'dotted black': 6, 'faded blue': 5}}
        lp = LuggageProcessing(self.input_data)
        self.assertEqual(lp.data, exp_out)
        
    def test_LuggageProcessing(self):
        exp_out = 4
        lp = LuggageProcessing(self.input_data)
        self.assertEqual(lp.get_bag_host_count('shiny gold'), exp_out)
        
unittest.main(argv=[""], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


In [2]:
lp = LuggageProcessing("input/d07.txt")
lp.get_bag_host_count('shiny gold', output=True)

### A `shiny gold` bag can be in `300` other bags

300

## \-\-\- Part Two \-\-\-

It's getting pretty expensive to fly these days \- not because of ticket prices, but because of the ridiculous number of bags you need to buy!

Consider again your `shiny gold` bag and the rules from the above example:

*   `faded blue` bags contain `0` other bags.
*   `dotted black` bags contain `0` other bags.
*   `vibrant plum` bags contain `11` other bags: 5 `faded blue` bags and 6 `dotted black` bags.
*   `dark olive` bags contain `7` other bags: 3 `faded blue` bags and 4 `dotted black` bags.

So, a single `shiny gold` bag must contain 1 `dark olive` bag (and the 7 bags within it) plus 2 `vibrant plum` bags (and the 11 bags within **each** of those): `1 + 1*7 + 2 + 2*11` = **`32`** bags!

Of course, the actual rules have a small chance of going several levels deeper than this example; be sure to count all of the bags, even if the nesting becomes topologically impractical!

Here's another example:

```
shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.

```

In this example, a single `shiny gold` bag must contain **`126`** other bags.

**How many individual bags are required inside your single `shiny gold` bag?**

In [3]:
class LuggageProcessing2(LuggageProcessing):
    
    debug = False
    
    def get_bag_count(self, target_bag, output=False):
        """A recursive function to get the total number of bags that will fit in the target_bag"""
        tbag_contents = self.data[target_bag]
        bag_count = 0
        if tbag_contents:
            for bag in tbag_contents:
                num_bags = tbag_contents[bag]
                contents = self.get_bag_count(bag)
                if contents > 0:
                    bag_count += num_bags * contents + num_bags
                else:
                    bag_count += num_bags
                    
            if self.debug:
                print(f'{target_bag} is worth {bag_count} and contains {tbag_contents}')
        
        if output:
            display(Markdown(f'### A `{target_bag}` is required to contain `{bag_count}` bags'))

        return bag_count


class TestBasic(unittest.TestCase):
        
    def test_LuggageProcessing2(self):
        input_data = ['light red bags contain 1 bright white bag, 2 muted yellow bags.\ndark orange bags contain 3 bright white bags, 4 muted yellow bags.\nbright white bags contain 1 shiny gold bag.\nmuted yellow bags contain 2 shiny gold bags, 9 faded blue bags.\nshiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.\ndark olive bags contain 3 faded blue bags, 4 dotted black bags.\nvibrant plum bags contain 5 faded blue bags, 6 dotted black bags.\nfaded blue bags contain no other bags.\ndotted black bags contain no other bags.',
                      'shiny gold bags contain 2 dark red bags.\ndark red bags contain 2 dark orange bags.\ndark orange bags contain 2 dark yellow bags.\ndark yellow bags contain 2 dark green bags.\ndark green bags contain 2 dark blue bags.\ndark blue bags contain 2 dark violet bags.\ndark violet bags contain no other bags.']
        exp_output = [32,126]
        for in_data, exp_out in tuple(zip(input_data, exp_output)):
            lp2 = LuggageProcessing2(in_data)
            self.assertEqual(lp2.get_bag_count('shiny gold'), exp_out)
        
unittest.main(argv=[""], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [4]:
lp2 = LuggageProcessing2("input/d07.txt")
lp2.get_bag_count('shiny gold', output=True)

### A `shiny gold` is required to contain `8030` bags

8030